In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import accuracy_score


# Загрузка данных
flats_data_without_renovation = pd.read_excel('flats.xlsx')
flats_data_with_renovation = pd.read_excel("euro_renovation.xlsx")

# Проверка типов данных в столбцах и преобразование строковых значений в числовой формат
target_column = '**price_sq:** Цена за квадратный метр.'
if flats_data_without_renovation[target_column].dtype == 'object':
    flats_data_without_renovation[target_column] = pd.to_numeric(flats_data_without_renovation[target_column], errors='coerce')
    flats_data_with_renovation[target_column] = pd.to_numeric(flats_data_with_renovation[target_column], errors='coerce')

# Удаление строк с отсутствующими значениями в целевой переменной
flats_data_with_renovation.dropna(subset=[target_column], inplace=True)

# Заполнение отсутствующих значений в остальных столбцах
flats_data_without_renovation.ffill(inplace=True)
flats_data_with_renovation.ffill(inplace=True)


# Определение целевой переменной
target_column = '**price_sq:** Цена за квадратный метр.'

# Выбор признаков и целевой переменной
X_with_renovation = flats_data_with_renovation.drop([target_column], axis=1)
y_with_renovation = flats_data_with_renovation[target_column]

# Создание дамми-переменных для категориальных признаков
X_with_renovation_encoded = pd.get_dummies(X_with_renovation)

# Разделение данных на обучающий и тестовый наборы
X_train_encoded, X_test_encoded, y_train, y_test = train_test_split(X_with_renovation_encoded, y_with_renovation, test_size=0.2, random_state=42)

# Создание объекта SimpleImputer и заполнение отсутствующих значений
imputer = SimpleImputer(strategy='mean')
imputer.fit(X_train_encoded)  # Обучение только на обучающем наборе

# Преобразование данных
X_train_encoded_imputed = imputer.transform(X_train_encoded)
X_test_encoded_imputed = imputer.transform(X_test_encoded)


# Обучение модели с евроремонтом на закодированных данных с заполнением отсутствующих значений
model = RandomForestRegressor(n_estimators=100, oob_score=True, random_state=100)
model.fit(X_train_encoded_imputed, y_train)




# Оценка модели
y_pred = model.predict(X_test_encoded_imputed)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Метрики для модели с ремонтом:")
print("Средняя абсолютная ошибка:", round(mae,3))
print("Среднеквадратичная ошибка:", round(mse,3))
print("R^2 Score:", round(r2,3))


# Прогнозирование на данных без евроремонта
flats_data_without_renovation_features = flats_data_without_renovation.drop([target_column], axis=1)
flats_data_without_renovation_features_encoded = pd.get_dummies(flats_data_without_renovation_features)

# Убедимся, что только известные признаки используются
missing_features = set(X_train_encoded.columns) - set(flats_data_without_renovation_features_encoded.columns)
if missing_features:
    flats_data_without_renovation_features_encoded = flats_data_without_renovation_features_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)

flats_data_without_renovation_features_encoded_imputed = imputer.transform(flats_data_without_renovation_features_encoded)
predicted_prices_without_renovation = model.predict(flats_data_without_renovation_features_encoded_imputed)

# Визуализация результатов и дальнейшие шаги (по вашему выбору)


C:\Users\aayza\AppData\Local\Temp\ipykernel_17276\2162555933.py:29: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  flats_data_with_renovation.ffill(inplace=True)


Метрики для модели с ремонтом:
Средняя абсолютная ошибка: 23211.874
Среднеквадратичная ошибка: 1155942051.828
R^2 Score: 0.854
OOB оценка качества модели: 0.829
